In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 4.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 4.9MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [7]:
cd /content/drive/My Drive/Colab Notebooks/Matrix2/data

/content/drive/My Drive/Colab Notebooks/Matrix2/data


In [9]:
df = pd.read_hdf('/content/drive/My Drive/Colab Notebooks/Matrix2/data/car.h5')
print (df.shape)
df.sample(3)

(106494, 155)


,breadcrumb,created_at,price_currency,price_details,price_value,seller_address,seller_name,seller_type,feature_czujniki-parkowania-przednie,feature_poduszka-powietrzna-chroniąca-kolana,feature_kurtyny-powietrzne,feature_klimatyzacja-dwustrefowa,feature_światła-led,feature_czujnik-zmierzchu,feature_elektrycznie-ustawiane-lusterka,feature_asr-(kontrola-trakcji),feature_poduszka-powietrzna-kierowcy,feature_cd,feature_elektryczne-szyby-przednie,feature_poduszka-powietrzna-pasażera,feature_system-start-stop,feature_światła-do-jazdy-dziennej,feature_komputer-pokładowy,feature_elektryczne-szyby-tylne,feature_klimatyzacja-manualna,feature_tapicerka-welurowa,feature_czujnik-deszczu,feature_światła-przeciwmgielne,feature_ogrzewanie-postojowe,feature_radio-niefabryczne,feature_regulowane-zawieszenie,feature_ogranicznik-prędkości,feature_zmieniarka-cd,feature_szyberdach,feature_isofix,feature_asystent-pasa-ruchu,feature_alufelgi,feature_bluetooth,feature_nawigacja-gps,feature_asystent-parkowania,...,param_particle-filter,param_zarejestrowany-w-polsce,param_oferta-od,param_moc,param_make,param_kraj-pochodzenia,param_engine-power,param_year,param_serwisowany-w-aso,param_rodzaj-paliwa,param_możliwość-finansowania,param_registered-in-poland,param_transmission,param_miesięczna-rata,param_kod-silnika,param_version,param_opłata-początkowa,param_model-pojazdu,param_pojemność-skokowa,param_kierownica-po-prawej-(anglik),param_fuel-type,param_vat-free,param_acrylic,param_damaged,param_door-count,param_tuning,param_filtr-cząstek-stałych,param_nr-of-seats,param_kolor,param_engine-capacity,param_pearl,param_stan,param_wersja,param_emisja-co2,param_body-type,param_matowy,param_bezwypadkowy,param_akryl-(niemetalizowany),param_monthly-payment-value,car_id
76797,"[Osobowe, Audi, A4, Audi A4 B5 (1995-2001)]","15:48, 2 marca 2018",PLN,"Cena Brutto, Do negocjacji",6200.0,"Wolbrom, olkuski, Małopolskie",None,Osoba prywatna,False,False,False,False,False,False,True,False,True,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,None,Tak,Osoby prywatnej,100 KM,None,None,None,None,None,Benzyna+LPG,None,None,None,None,None,None,None,A4,1 595 cm3,None,None,None,None,None,None,None,None,None,Inny kolor,None,None,Używane,B5 (1995-2001),None,None,None,None,None,None,76797
7000,"[Osobowe, Lexus, IS, Lexus IS I (1999-2005)]","13:25, 2 marca 2018",PLN,Cena Brutto,14700.0,"Sosnowiec, Śląskie",None,Osoba prywatna,False,False,True,False,False,False,True,True,True,True,True,True,False,False,True,True,False,True,False,True,False,False,False,False,False,False,False,False,True,True,False,False,...,None,Tak,Osoby prywatnej,155 KM,None,None,None,None,None,Benzyna,None,None,None,None,None,None,None,IS,1 988 cm3,None,None,None,None,None,None,None,None,None,Szary,None,None,Używane,I (1999-2005),None,None,None,None,None,None,7000
10938,"[Osobowe, Škoda, Octavia, Škoda Octavia I (199...","23:40, 5 marca 2018",PLN,"Cena Brutto, Do negocjacji",11500.0,"ul.Bielska 63 - 09-400 Płock, Mazowieckie (Pol...",JACKPOL AUTO samochody używane z GWARANCJĄ,Dealer,False,False,True,False,False,False,True,True,True,True,True,True,False,False,True,False,True,True,True,True,False,True,False,False,False,False,True,False,False,False,False,False,...,None,None,Firmy,101 KM,None,Belgia,None,None,Tak,Diesel,Tak,None,None,None,None,None,None,Octavia,1 900 cm3,None,None,None,None,None,None,None,None,None,Srebrny,None,None,Używane,I (1996-2011),None,None,None,Tak,None,None,10938


In [10]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values


cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
print (len(cat_feats))



151


In [11]:
X = df[cat_feats].values
y = df['price_value'].values


model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')

np.mean(scores), np.std(scores)

(-19695.13091100928, 148.72570644015792)

In [13]:
def run_model (model, feats):
  X = df[feats].values
  y = df['price_value'].values


  # model = DecisionTreeRegressor(max_depth=5)
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')

  return np.mean(scores), np.std(scores)


run_model(DecisionTreeRegressor(max_depth=5), cat_feats )

(-19695.13091100928, 148.72570644015937)

## Decision Tree

In [16]:
model = DecisionTreeRegressor(max_depth=5)
run_model(model, cat_feats )

(-19695.13091100928, 148.72570644015792)

## Random Forest

In [17]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=2020)
run_model(model, cat_feats )

(-18774.078311945963, 56.07799419652943)

## XGBBoost

In [18]:
xgb_params = {
    'max_depth': 5,
    'n_estimators':50,
    'learning_rate': 0.1, 
    'seed': 0
}

model = xgb.XGBRegressor(**xgb_params)
run_model(model, cat_feats )

[04:56:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:57:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:57:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [19]:
m = xgb.XGBRegressor(**xgb_params)
m.fit (X,y,)

imp = PermutationImportance(m, random_state=2020).fit(X,y)
eli5.show_weights(m, feature_names=cat_feats)

[05:00:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.0951,param_faktura-vat__cat
0.0949,feature_kamera-cofania__cat
0.0669,feature_łopatki-zmiany-biegów__cat
0.0664,param_napęd__cat
0.0560,param_skrzynia-biegów__cat
0.0423,feature_asystent-pasa-ruchu__cat
0.0411,param_stan__cat
0.0329,feature_światła-led__cat
0.0279,feature_bluetooth__cat
0.0263,feature_regulowane-zawieszenie__cat


In [0]:
feats = [
  'feature_kamera-cofania__cat', 
  'feature_łopatki-zmiany-biegów__cat', 
  'param_napęd__cat', 
  'param_skrzynia-biegów__cat', 
  'param_faktura-vat__cat', 
  'feature_asystent-pasa-ruchu__cat', 
  'param_stan__cat', 
  'feature_światła-led__cat', 
  'feature_bluetooth__cat',
  'feature_regulowane-zawieszenie__cat', 
  'feature_wspomaganie-kierownicy__cat', 
  'feature_system-start-stop__cat', 
  'feature_światła-do-jazdy-dziennej__cat', 
  'feature_światła-xenonowe__cat', 
  'feature_czujniki-parkowania-przednie__cat', 
  'param_moc__cat', 
  'param_rok-produkcji__cat', 
  'param_pojemność-skokowa__cat', 
  'feature_asystent-parkowania__cat', 
  'seller_name__cat']

In [31]:
model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats )

[05:18:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:18:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:18:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-14242.8316835859, 148.71839978475074)

In [32]:
df['param_rok-produkcji'].unique()

array(['2018', '2011', '2015', '2009', '2017', '2012', '2013', '2007',
       '2001', '2016', '2006', '2008', '2004', '1999', '2000', '2010',
       '2005', '2002', '1998', '2014', '2003', '1982', '1995', '1997',
       '1992', '1993', '1994', '1996', '1989', '1988', '1967', '1987',
       '1970', '1959', '1990', '1991', '1974', None, '1975', '1973',
       '1953', '1985', '1984', '1986', '1981', '1979', '1960', '1983',
       '1978', '1964', '1980', '1972', '1969', '1956', '1966', '1977',
       '1962', '1965', '1971', '1963', '1961', '1952', '1949', '1976',
       '1937', '1968', '1958', '1955', '1933', '1929', '1957', '1944',
       '1954', '1932', '1936', '1947', '1948'], dtype=object)

In [36]:
df['param_rok-produkcji'] = df['param_rok-produkcji']. map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_rok-produkcji'].unique()

array([2018, 2011, 2015, 2009, 2017, 2012, 2013, 2007, 2001, 2016, 2006,
       2008, 2004, 1999, 2000, 2010, 2005, 2002, 1998, 2014, 2003, 1982,
       1995, 1997, 1992, 1993, 1994, 1996, 1989, 1988, 1967, 1987, 1970,
       1959, 1990, 1991, 1974,   -1, 1975, 1973, 1953, 1985, 1984, 1986,
       1981, 1979, 1960, 1983, 1978, 1964, 1980, 1972, 1969, 1956, 1966,
       1977, 1962, 1965, 1971, 1963, 1961, 1952, 1949, 1976, 1937, 1968,
       1958, 1955, 1933, 1929, 1957, 1944, 1954, 1932, 1936, 1947, 1948])

In [37]:
feats = [
  'feature_kamera-cofania__cat', 
  'feature_łopatki-zmiany-biegów__cat', 
  'param_napęd__cat', 
  'param_skrzynia-biegów__cat', 
  'param_faktura-vat__cat', 
  'feature_asystent-pasa-ruchu__cat', 
  'param_stan__cat', 
  'feature_światła-led__cat', 
  'feature_bluetooth__cat',
  'feature_regulowane-zawieszenie__cat', 
  'feature_wspomaganie-kierownicy__cat', 
  'feature_system-start-stop__cat', 
  'feature_światła-do-jazdy-dziennej__cat', 
  'feature_światła-xenonowe__cat', 
  'feature_czujniki-parkowania-przednie__cat', 
  'param_moc__cat', 
  'param_rok-produkcji', 
  'param_pojemność-skokowa__cat', 
  'feature_asystent-parkowania__cat', 
  'seller_name__cat']

model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats )

[05:26:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:26:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:26:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-12606.970031747682, 70.36483448003113)

In [48]:
df['param_moc'].unique()

array([ 90, 115, 262, 110, 310, 105, 140, 175, 125, 185, 190, 440, 141,
       200, 224,  75,  99, 184, 109, 233, 116,  68, 286, 126, 160, 135,
       120, 272,  -1, 150, 180, 136, 102, 131, 218, 245, 170, 112, 250,
       252,  73, 100, 313, 101, 285,  70, 383, 174, 277, 132, 130, 215,
        60, 330, 163, 177,  98,  78, 189, 156, 143,  69, 113,  65, 122,
        82, 251,  95, 197, 235, 238, 171, 381, 400, 178,  80, 165,  85,
       258, 142, 204, 124,  55, 144, 231, 248, 152, 181, 210, 340, 129,
       147,  50,  54, 290, 306, 193,  77, 164,  96, 194, 111, 166, 206,
       118, 360, 211, 271, 455, 280, 106, 114, 421,  74, 213, 121, 275,
       435, 384, 326,  88, 220, 260,  64,  86, 128, 256, 240, 244, 162,
       237, 350,  35, 265, 202, 133,  83, 117, 146,  92, 192, 145, 525,
       254, 182, 328, 367, 148, 456,  97, 270, 107, 108, 203, 155,  94,
        93, 241,  20,  71, 173,  58, 205, 236,   1, 557,  84, 457,  72,
       295, 134, 425, 228,  81, 230, 201,  87, 234, 299, 585, 20

In [46]:


df['param_moc'] = df['param_moc']. map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0])) #    ||-1 if str(x) == 'None' else int(x))
df['param_moc'].unique()

array([ 90, 115, 262, 110, 310, 105, 140, 175, 125, 185, 190, 440, 141,
       200, 224,  75,  99, 184, 109, 233, 116,  68, 286, 126, 160, 135,
       120, 272,  -1, 150, 180, 136, 102, 131, 218, 245, 170, 112, 250,
       252,  73, 100, 313, 101, 285,  70, 383, 174, 277, 132, 130, 215,
        60, 330, 163, 177,  98,  78, 189, 156, 143,  69, 113,  65, 122,
        82, 251,  95, 197, 235, 238, 171, 381, 400, 178,  80, 165,  85,
       258, 142, 204, 124,  55, 144, 231, 248, 152, 181, 210, 340, 129,
       147,  50,  54, 290, 306, 193,  77, 164,  96, 194, 111, 166, 206,
       118, 360, 211, 271, 455, 280, 106, 114, 421,  74, 213, 121, 275,
       435, 384, 326,  88, 220, 260,  64,  86, 128, 256, 240, 244, 162,
       237, 350,  35, 265, 202, 133,  83, 117, 146,  92, 192, 145, 525,
       254, 182, 328, 367, 148, 456,  97, 270, 107, 108, 203, 155,  94,
        93, 241,  20,  71, 173,  58, 205, 236,   1, 557,  84, 457,  72,
       295, 134, 425, 228,  81, 230, 201,  87, 234, 299, 585, 20

In [47]:
feats = [
  'feature_kamera-cofania__cat', 
  'feature_łopatki-zmiany-biegów__cat', 
  'param_napęd__cat', 
  'param_skrzynia-biegów__cat', 
  'param_faktura-vat__cat', 
  'feature_asystent-pasa-ruchu__cat', 
  'param_stan__cat', 
  'feature_światła-led__cat', 
  'feature_bluetooth__cat',
  'feature_regulowane-zawieszenie__cat', 
  'feature_wspomaganie-kierownicy__cat', 
  'feature_system-start-stop__cat', 
  'feature_światła-do-jazdy-dziennej__cat', 
  'feature_światła-xenonowe__cat', 
  'feature_czujniki-parkowania-przednie__cat', 
  'param_moc', 
  'param_rok-produkcji', 
  'param_pojemność-skokowa__cat', 
  'feature_asystent-parkowania__cat', 
  'seller_name__cat']

model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats )

[05:40:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:40:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:40:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10668.21096354992, 65.60418228047648)

In [49]:
df['param_pojemność-skokowa'].unique()

array(['898 cm3', '1 560 cm3', '3 000 cm3', ..., '5 992 cm3', '1 966 cm3',
       '142 280 cm3'], dtype=object)

In [54]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')))
df['param_pojemność-skokowa'].unique()

array([   898,   1560,   3000, ...,   5992,   1966, 142280])

In [55]:
feats = [
  'feature_kamera-cofania__cat', 
  'feature_łopatki-zmiany-biegów__cat', 
  'param_napęd__cat', 
  'param_skrzynia-biegów__cat', 
  'param_faktura-vat__cat', 
  'feature_asystent-pasa-ruchu__cat', 
  'param_stan__cat', 
  'feature_światła-led__cat', 
  'feature_bluetooth__cat',
  'feature_regulowane-zawieszenie__cat', 
  'feature_wspomaganie-kierownicy__cat', 
  'feature_system-start-stop__cat', 
  'feature_światła-do-jazdy-dziennej__cat', 
  'feature_światła-xenonowe__cat', 
  'feature_czujniki-parkowania-przednie__cat', 
  'param_moc', 
  'param_rok-produkcji', 
  'param_pojemność-skokowa', 
  'feature_asystent-parkowania__cat', 
  'seller_name__cat']

model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats )

[06:02:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:02:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:02:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10418.18729276653, 72.80569459652894)